# 3. Implementierung der Hauptkomponentenanalyse

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
%matplotlib inline
%load_ext version_information

## a) Implementieren Sie ein Python-Modul, das eine Funktion zur Hauptkomponentenanalyse zur Verfügung stellt.

In [ ]:
def pca(data):

    # Schritt 1.a: Mittelwert jedes Merkmals berechnen
    num_data_points = len(data)
    num_features = len(data[0])
    mean_values = [0.0] * num_features
    for i in range(num_data_points):
        for j in range(num_features):
            mean_values[j] += data[i][j]
    for j in range(num_features):
        mean_values[j] /= num_data_points

    # Schritt 1.b: Daten zentrieren
    centered_data = []

    for i in range(num_data_points):
        centered_point = []
        for j in range(num_features):
            centered_point.append(data[i][j] - mean_values[j])
        centered_data.append(centered_point)

    # Schritt 2: Daten normalisieren
    normalized_data = []
    min_values = [min(col) for col in zip(*centered_data)]
    max_values = [max(col) for col in zip(*centered_data)]

    for i in range(num_data_points):
        normalized_point = []
        for j in range(num_features):
            normalized_value = (centered_data[i][j] - min_values[j]) / (max_values[j] - min_values[j])
            normalized_point.append(normalized_value)
        normalized_data.append(normalized_point)

    # Schritt 3: Designmatrix erstellen
    design_matrix = np.array(normalized_data)

    # Schritt 4: Singulaerwertzerlegung von X berechnen
    return np.linalg.svd(data)

## b) Testen Sie Ihr Modul innerhalb eines IPython-Notebooks am Datensatz Boston Housing.
### Lassen Sie dabei die Variable TGT weg. Stellen Sie Ihre Ergebnisse in einer Tabelle mit den Eigenwerten der Kovarianzmatrix (Achtung: die Diagonalelemente von müssen dafür quadriert und durch n − 1 geteilt werden. Warum?).
-> Ist notwendig, um eine konsistente Schätzung der Kovarianzmatrix zu erhalten, wenn man mit Stichproben arbeitet.

In [ ]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data'
cols = ['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B',
        'LSTAT','TGT']
boston = pd.read_csv(url, sep=' ', skipinitialspace=True, header=None, names=cols, 
                     index_col=False)

TGT = boston['TGT']
del boston['TGT']
data = boston.values.tolist()

X = pca(data)
U, D, Vt = X

D = np.diag(D)

n = len(D)
a = {}
for i in range(n):
    a[i] = D[i][i]*D[i][i]/(n-1)
a

### dem Anteil der zugehörigen Hauptkomponente an an der Gesamtvarianz (“erklärte Varianz”)

In [ ]:
s = sum(a)
norm = [float(i)/s for i in a]
norm

### und der kumulativen erklärten Varianz dar, d.h. welchen Varianzanteil die ersten Komponenten zusammen erklären.

In [ ]:
np.cumsum(norm)

### Wieviele Dimensionen können Sie weglassen, wenn Sie 10%, 5% und 1% Fehler bei der Dimensionsreduktion zulassen?
10% - 4 Dimensionen
5% - 3 Dimensionen
1% - 2 Dimensionen

## c) Berechnen Sie die Matrix der Korrelationskoeffizienten für die transformierten Variablen und interpretieren Sie das Ergebnis.
Korrelieren sehr schwach. Macht Sinn, ist schließlich der Zweck der PCA.

In [ ]:
def color_yes_no(v):
    return f"color: green;" if v > 0.5 else f"color: red;"
df = pd.DataFrame(pd.DataFrame(Vt).corr().abs())
df.style.map(color_yes_no)

## d.1) Berechnen Sie den Korrelationskoeffizienten der Projektionen auf die ersten drei Hauptkomponenten mit den ursprünglichen Variablen. 

In [ ]:
PC1 = U[:, 0]
PC2 = U[:, 1]
PC3 = U[:, 2]
correlation_PC1 = np.corrcoef(PC1, data, rowvar=False)[0, 1:]
correlation_PC2 = np.corrcoef(PC2, data, rowvar=False)[0, 1:]
correlation_PC3 = np.corrcoef(PC3, data, rowvar=False)[0, 1:]
print(correlation_PC1)
print(correlation_PC2)
print(correlation_PC3)

## d.2) Interpretieren Sie Ihr Ergebnis.

Viele hohe Korrelationen.
PC1 und PC2 korrelieren teilweise mit den gleichen Variablen, allerdings mit verschiedenem Vorzeichen.

## e.1) Stellen Sie die ersten beiden der neuen Variablen als Scatterplot dar (am besten in Pandas-Dataframe importieren). Plotten Sie dabei alle Datenpunkte mit einem Hauspreis oberhalb des Medians aller Hauspreise in einer anderen Farbe als die Datenpunkte unterhalb.

In [ ]:
median_tgt = TGT.median()
col = np.where(TGT>median_tgt,'b','r')
plt.scatter(PC1, TGT, c = col)
plt.xlabel('PC1')
plt.ylabel('TGT')
plt.show()

In [ ]:
plt.scatter(PC2, TGT, c = col)
plt.xlabel('PC2')
plt.ylabel('TGT')
plt.show()

## e.2) Eignen sich die beiden neuen Variablen zur Vorhersage des Hauspreises?

Nein. Der Hauspreis streut sehr stark bei bestimmten Werten der Variablen.

In [ ]:
%version_information